#### Код из методички

Исходный код реализации дерева решений

In [1]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np
import math

In [2]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_samples = 5000, n_features = 10, n_informative = 5, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=5)

In [3]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [4]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction        

In [5]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [6]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [7]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [8]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [9]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [11]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [12]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [13]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels)

In [14]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")
    
print_tree(my_tree)

Индекс 4
Порог 0.050912454169529386
--> True:
  Индекс 1
  Порог -0.6746805049117537
  --> True:
    Индекс 9
    Порог -2.1577953048777267
    --> True:
      Индекс 2
      Порог 1.610144274992387
      --> True:
        Прогноз: 1
      --> False:
        Прогноз: 1
    --> False:
      Индекс 8
      Порог 1.9674685119070623
      --> True:
        Индекс 2
        Порог 2.0973544150678216
        --> True:
          Индекс 1
          Порог -0.8998816153257458
          --> True:
            Прогноз: 0
          --> False:
            Индекс 6
            Порог 0.39168435348668373
            --> True:
              Прогноз: 0
            --> False:
              Прогноз: 0
        --> False:
          Прогноз: 0
      --> False:
        Прогноз: 0
  --> False:
    Индекс 3
    Порог -3.1210238108868946
    --> True:
      Индекс 0
      Порог 0.1331926254766348
      --> True:
        Прогноз: 0
      --> False:
        Прогноз: 0
    --> False:
      Индекс 1
      Порог -0.1083

----

#### Задание 1.
*В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).*

----

Реализуем критерий останова по количеству использованных признаков (max_params) и глубине дерева (max_depth).

Для этого превратим build_tree в класс.

In [15]:
# Класс дерева
class Tree:

    def __init__(self, max_params = 40, max_depth=40, criteria = "gini"):
        self.max_params = max_params
        self.max_depth = max_depth
        self.criteria = criteria
        if criteria == 'entropy':
            self.criteriaFunc = self.entropy
        else:
            self.criteriaFunc = self.gini
        
        self.params = []
        
    # Обучение
    def fit(self, data, labels):
        self.tree = self.build(data, labels)
        return self.tree
    
    # предсказание
    def predict(self, data):
    
        classes = []
        for obj in data:
            prediction = self.classify(obj, self.tree)
            classes.append(prediction)
        return classes
        
    # Построение дерева
    def build(self, data, labels, depth=1):
        
        # останов по глубине дерева
        if depth>self.max_depth:
            return Leaf(data, labels)
        
        quality, t, index = self.find_split(data, labels)

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if quality == 0:
            return Leaf(data, labels)
        
        # добавляем признак
        if not(index in self.params):
            self.params.append(index)
            
        # останов по использованным признакам
        if len(self.params)>self.max_params:
            return Leaf(data, labels)

        true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)

        # Рекурсивно строим два поддерева
        true_branch = self.build(true_data, true_labels, depth+1)
        false_branch = self.build(false_data, false_labels, depth+1)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)
    
    # Нахождение наилучшего разбиения
    def find_split(self, data, labels):
        #  обозначим минимальное количество объектов в узле
        min_leaf = 5

        current_crit = self.criteriaFunc(labels)

        best_quality = 0
        best_t = None
        best_index = None

        n_features = data.shape[1]

        for index in range(n_features):
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in data])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                #  пропускаем разбиения, в которых в узле остается менее 5 объектов
                if len(true_data) < min_leaf or len(false_data) < min_leaf:
                    continue

                current_quality = self.quality(true_labels, false_labels, current_crit)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index
    
    # Разбиение датасета в узле
    def split(self, data, labels, index, t):

        left = np.where(data[:, index] <= t)
        right = np.where(data[:, index] > t)

        true_data = data[left]
        false_data = data[right]
        true_labels = labels[left]
        false_labels = labels[right]

        return true_data, false_data, true_labels, false_labels
    
    # предсказание
    def classify(self, obj, node):

        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.classify(obj, node.true_branch)
        else:
            return self.classify(obj, node.false_branch)
        
    # Расчет качества
    def quality(self, left_labels, right_labels, current_crit):

        # доля выбоки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

        return current_crit - p * self.criteriaFunc(left_labels) - (1 - p) * self.criteriaFunc(right_labels)
    
    # критерий Джини
    def gini(self, labels):
        #  подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        #  расчет критерия
        impurity = 1
        for label in classes:
            p = classes[label] / len(labels)
            impurity -= p ** 2
        return impurity
    
    # критерий Энтропии
    def entropy(self, labels):
        #  подсчет количества объектов разных классов
        classes = {}
        for label in labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1

        #  расчет критерия
        impurity = 0
        for label in classes:
            p = classes[label] / len(labels)
            if p!=0:
                impurity -= p * math.log(p, 2)
        return impurity

In [16]:
# Построим дерево по обучающей выборке с ограничениями
tm = Tree(max_depth=3)
treeLimit = tm.build(train_data, train_labels)
print_tree(treeLimit)

Индекс 4
Порог 0.050912454169529386
--> True:
  Индекс 1
  Порог -0.6746805049117537
  --> True:
    Индекс 9
    Порог -2.1577953048777267
    --> True:
      Прогноз: 1
    --> False:
      Прогноз: 0
  --> False:
    Индекс 3
    Порог -3.1210238108868946
    --> True:
      Прогноз: 0
    --> False:
      Прогноз: 1
--> False:
  Индекс 9
  Порог -1.4995309772766199
  --> True:
    Индекс 4
    Порог 0.7081766640015936
    --> True:
      Прогноз: 1
    --> False:
      Прогноз: 0
  --> False:
    Индекс 1
    Порог 0.8942941373716249
    --> True:
      Прогноз: 0
    --> False:
      Прогноз: 0


----
#### Задание 2. 
*Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.*

----

In [17]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

tgini = Tree()
tgini.fit(train_data, train_labels)
train_predict = tgini.predict(train_data)
test_predict = tgini.predict(test_data)

accuracy_metric(train_labels, train_predict), accuracy_metric(test_labels, test_predict)

(99.02857142857144, 97.93333333333332)

In [18]:
tentropy = Tree(criteria='entropy')
tentropy.fit(train_data, train_labels)
train_predict = tentropy.predict(train_data)
test_predict = tentropy.predict(test_data)

print_tree(tentropy.tree)
accuracy_metric(train_labels, train_predict), accuracy_metric(test_labels, test_predict)

Индекс 4
Порог 0.1742105819604569
--> True:
  Индекс 1
  Порог -0.6746805049117537
  --> True:
    Индекс 9
    Порог -2.1577953048777267
    --> True:
      Индекс 2
      Порог 1.610144274992387
      --> True:
        Прогноз: 1
      --> False:
        Прогноз: 1
    --> False:
      Индекс 6
      Порог 0.9074349785618837
      --> True:
        Прогноз: 0
      --> False:
        Индекс 5
        Порог -1.2788632947561636
        --> True:
          Прогноз: 0
        --> False:
          Индекс 2
          Порог 0.16114667302030877
          --> True:
            Прогноз: 0
          --> False:
            Индекс 5
            Порог -0.5806660172131473
            --> True:
              Прогноз: 0
            --> False:
              Прогноз: 0
  --> False:
    Индекс 1
    Порог 0.08764164038484101
    --> True:
      Индекс 9
      Порог -1.2719718607957082
      --> True:
        Индекс 2
        Порог -1.315844598197269
        --> True:
          Индекс 2
          Порог -

(99.11428571428571, 98.13333333333333)

При малой выборке (100 элементов) качество классификации и деревья, которые были построены были идентичны.

При увеличении выборки (до 1000 и 5000 элементов) качество классификации стало незначительно лучше у энтропийного критерия, но лишь на десятые доли процента.

Оба критерия позволяют достичь хорошего результата и сопостовимы друг с другом по качетсву.

----
#### Задание 3.
*Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.*

----

In [55]:
# дерево для задачи регрессии
class RegressionTree(Tree):
    def __init__(self, max_params = 40, max_depth=40, criteria = "gini"):
        self.max_params = max_params
        self.max_depth = max_depth
        self.criteria = criteria
        self.criteriaFunc = self.std
        
        self.params = []
        
    # Дисперсия
    def std(self, labels):
        #  подсчет количества объектов разных классов
        mean = np.mean(labels)
        
        #  расчет критерия
        impurity = 0
        for label in labels:
            impurity += (label-mean)**2 / len(labels)
        return impurity
    
    # предсказание
    def predict(self, data):
    
        predictions = []
        for obj in data:
            prediction = self.getTarget(obj, self.tree)
            predictions.append(prediction)
        return predictions
    
    # предсказание
    def getTarget(self, obj, node):

        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = np.mean(node.labels)
            return answer

        if obj[node.index] <= node.t:
            return self.getTarget(obj, node.true_branch)
        else:
            return self.getTarget(obj, node.false_branch)

In [61]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

# сгенерируем данные
data, target = datasets.make_regression(n_samples=1000, n_features = 2, n_targets = 1, 
                                              noise = 1, random_state = 27)
train_X, test_X, train_y, test_y = model_selection.train_test_split(data, target, test_size = 0.3, random_state = 1)

regTree = RegressionTree()
regTree.fit(train_X, train_y)
train_predict = regTree.predict(train_X)
test_predict = regTree.predict(test_X)

calc_mse(train_y, train_predict), calc_mse(test_y, test_predict)

(55.885066164658795, 130.58226915590203)